In [2]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.1MB/s 


In [13]:
import pandas as pd
import matplotlib as plt
import os
import csv
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

from skmultilearn.problem_transform import LabelPowerset
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, cross_val_score,StratifiedKFold
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,precision_score,recall_score

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import NuSVC, LinearSVC, SVC

import pickle
from scipy.sparse import hstack
from skmultilearn.model_selection import iterative_train_test_split

In [4]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
dir = "drive/My Drive/BT4221/Data/"

# cleaned train dataset
df = pd.read_csv(dir+"cleaned_train.csv")
# for EDA
data_raw = pd.read_csv(dir+"train.csv")

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,word_density,total_length,capitals,capitals_prop,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,num_unique_words,prop_unique_words,processed_text,polarity,subjectivity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,43,223,0.191964,264,17,0.064394,0,1,6,0,41,0.953488,explanation edits made username hardcore metal...,0.136364,0.454545
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,17,95,0.177083,112,8,0.071429,1,0,5,0,17,1.000000,daww match background colour seemingly stuck t...,0.200000,0.200000
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,42,192,0.217617,233,4,0.017167,0,0,4,0,39,0.928571,hey man really trying edit war guy constantly ...,0.150000,0.383333
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,113,511,0.220703,622,11,0.017685,0,0,6,0,82,0.725664,ca nt make real suggestion improvement wondere...,0.183333,0.297222
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,13,55,0.232143,67,2,0.029851,0,1,3,0,13,1.000000,sir hero chance remember page,0.000000,0.000000


In [6]:
count_vect = pickle.load(open('/content/drive/My Drive/BT4221/Models/count_vect_v1.sav','rb'))
tf = pickle.load(open('/content/drive/My Drive/BT4221/Models/tf_v1.sav','rb'))

In [7]:
X= df['processed_text']
count_vect = CountVectorizer()
X_count = count_vect.fit_transform(X)
tf = TfidfTransformer()
X = tf.fit_transform(X_count)

# Iterative Stratification + Scaling

In [8]:
added_features = pd.DataFrame(df.iloc[:, 8:23])
added_features.drop(columns='processed_text',inplace=True)
for column in added_features.columns:
    var_sparse = StandardScaler().fit_transform(added_features[column].values[:, None])
    X = hstack((X, var_sparse.astype(float)),format='csr')
print(X.shape)

(159502, 188840)


In [9]:
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [11]:
clf_BR = pickle.load(open('/content/drive/My Drive/BT4221/Models/BR_SGD_v3.sav','rb'))
clf_CC = pickle.load(open('/content/drive/My Drive/BT4221/Models/CC_SGD_v3.sav','rb'))
clf_LP = pickle.load(open('/content/drive/My Drive/BT4221/Models/LP_SGD_v3.sav','rb'))

In [17]:
prediction_BR = clf_BR.predict(X_test)
prediction_CC = clf_CC.predict(X_test)
prediction_LP = clf_LP.predict(X_test)

ValueError: ignored

In [14]:
print("Recall (Binary Relevance) : {}".format(recall_score(prediction_BR,y_test,average='micro')))
print("Precision (Binary Relevance) : {}".format(precision_score(prediction_BR,y_test,average='micro')))

NameError: ignored

In [15]:
print("Recell (Classification Chain) : {}".format(recall_score(prediction_CC,y_test,average='micro')))
print("Precision (Classification Chain) : {}".format(precision_score(prediction_CC,y_test,average='micro')))

NameError: ignored

In [16]:
print("Recall (Label Powerset) : {}".format(recall_score(prediction_LP,y_test,average='micro')))
print("Precision (Label Powerset) : {}".format(precision_score(prediction_LP,y_test,average='micro')))

NameError: ignored